In [42]:
import numpy as np
from scipy import stats
import scipy.io as sio
from os import getcwd
from os.path import join 
import pandas as pd
from scipy.stats.stats import mode
from sklearn.model_selection import (train_test_split, RepeatedKFold, RandomizedSearchCV)
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from statsmodels.tsa.ar_model import AutoReg
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression 
from sklearn import svm
from sklearn import neighbors
import tensorflow as ts
from tensorflow.keras import layers   
from tensorflow import keras

In [18]:
fname = join(getcwd(),'data','data3SS2009.mat')

mat_contents = sio.loadmat(fname)

dataset = mat_contents['dataset']

N, Chno, Nc = dataset.shape
# N: number of samples
# Chno: number of channels
# Nc: number of cases

Y = mat_contents['labels'].reshape(Nc)

# print sizes 
pd.DataFrame(np.array([N, Chno, Nc,len(Y)]))

,0
0,8192
1,5
2,850
3,850


In [ ]:
# AutoRegression
X_ = np.empty((0, 850), float)
for i in range(4):
    chdata = dataset[:,i+1,:]
    ch_corr = []
    for j in range(np.shape(chdata)[1]):
        res = AutoReg(chdata[:,j],lags=29,old_names=False).fit()
        ch_corr.append(res.params)
    
    X_ = np.append(X_, np.array(ch_corr).transpose() , axis=0)
X = np.transpose(X_)

# np.shape(X)

# pca
pca = PCA(random_state=0, whiten=True, n_components=.95)
X_pca = pca.fit_transform(X)
Xpca_scaled = MinMaxScaler(feature_range=(-1,1)).fit_transform(X_pca)



# Setting the data

In [78]:
# convert class vectors to binary class matrices

# classes from 0-16 instead of 1-17
Yc = Y - np.ones(np.shape(Y))
X_train, X_test, Y_train, Y_test = train_test_split(Xpca_scaled,Yc,test_size=0.4,random_state=42)  
y_train = keras.utils.to_categorical(Y_train, num_classes=17)
y_test = keras.utils.to_categorical(Y_test, num_classes=17)

# Defining & training the Artificial Neural Net

In [81]:
# FEEDFORWARD MODEL
# model = ks.Sequential([
#   layers.Flatten(input_shape=np.shape(Xpca_scaled)[1]),
#   layers.Dense(32, activation='relu'),
#   layers.Dense(17, activation="softmax")
# ])

model = keras.Sequential([
  layers.Dense(8, input_shape=(np.shape(Xpca_scaled)[1],)),
  layers.Dense(14),
  layers.Dense(17, activation="softmax")
])

model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(0.01),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

model.fit(
    X_train, 
    y_train,
    epochs=14,
    validation_split=0.1,
)

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_101 (Dense)           (None, 8)                 56        
                                                                 
 dense_102 (Dense)           (None, 14)                126       
                                                                 
 dense_103 (Dense)           (None, 17)                255       
                                                                 
Total params: 437
Trainable params: 437
Non-trainable params: 0
_________________________________________________________________
Epoch 1/14
15/15 [==============================] - 1s 13ms/step - loss: 2.5287 - accuracy: 0.2244 - val_loss: 2.1154 - val_accuracy: 0.2941
Epoch 2/14
15/15 [==============================] - 0s 4ms/step - loss: 1.7809 - accuracy: 0.5142 - val_loss: 1.4122 - val_accuracy: 0.6078
Epoch 3/14
15/15 [====================

# Evaluating the model with the test data

In [82]:
score = model.evaluate(X_test, y_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

11/11 [==============================] - 0s 997us/step - loss: 0.1885 - accuracy: 0.9412
Test loss: 0.18853674829006195
Test accuracy: 0.9411764740943909


# Comparison with other models

Do exercício takehome5, temos os scores dos diversos métodos avaliados no curso

LReg - 0.944118 <br />
SVC  - 0.950000 <br />
kNN  - 0.911765 <br />
DTC  - 0.888235 <br />
RFC  - 0.929412 <br />
ANN  - 0.941176 <br />

Para os parâmetros utilizados, a ANN teve uma pior performance do que a classificação por regressão linear e SVC